# SDG Report

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
project_id = "ds-data-solutions-gbq"
%load_ext google.cloud.bigquery

Authenticated
The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Setup Parameters

In [ ]:
# eg for params
bq_params = {}
bq_params["gridid"] = "grid.4991.5"

## (A) Org SDG Report - overview stats

In [ ]:
%%bigquery --params $bq_params --project $project_id 
#StandardSQL

WITH pubs AS (
  SELECT 
    pub.id, 
    author_grid, 
    pub.year,
    IF('oa_all' in UNNEST(open_access_categories), True, False) as is_oa,
    IF(publication_type = 'preprint', True, False) as is_preprint,
  FROM 
    `dimensions-ai.data_analytics.publications` pub, UNNEST(pub.author_grid_ids) author_grid
  WHERE 
    @gridid in UNNEST(pub.author_grid_ids) 
    AND 
    (pub.year = 2009 OR pub.year = 2019)
    
), all_pubs AS (

  SELECT 
    pub.id, 
    pub.citations_count,
    ARRAY_LENGTH(pub.patents) as patent_citations
  FROM 
    `dimensions-ai.data_analytics.publications` pub
  WHERE 
    (pub.year >= 2009 AND pub.year <= 2020)
    AND 
    @gridid in UNNEST(pub.author_grid_ids)
    
), grants AS (

  SELECT 
    funding_amount_in_usd 
  FROM 
    `dimensions-ai.data_analytics.grants` 
  WHERE 
    @gridid in UNNEST(research_organisations) 
    AND 
    (start_year >= 2009 AND start_year <= 2020)
    
), patents AS (

  SELECT DISTINCT 
    id,
    ARRAY_LENGTH(publication_references) as pub_citations 
  FROM 
    `dimensions-ai.data_analytics.patents`
  WHERE
  	@gridid in UNNEST(assignee_grid_ids)
    AND 
    (publication_year >= 2009 AND publication_year <= 2020)
    
), clinical_trails_count AS (

  SELECT COUNT(*) FROM `dimensions-ai.data_analytics.clinical_trials` WHERE start_year >= 2009 AND start_year <= 2020 AND @gridid in UNNEST(sponsor_collaborators)
  
), data_sets_count AS (

  SELECT COUNT(*) FROM `dimensions-ai.data_general.data_sets` WHERE publication_year >= 2009 AND publication_year <= 2020 AND @gridid in UNNEST(research_orgs)

), pubs_count_2009 AS (

  SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2009

), pubs_count_2019 AS (

  SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2019
)


SELECT

  -- All pubs between 2009 and 2019
  (SELECT COUNT(*) FROM all_pubs) as total_pubs,

  -- All pubs 2009-2020 with SDG
  (
    SELECT COUNT(*) 
    FROM `dimensions-ai.data_analytics.publications` 
    WHERE @gridid in UNNEST(author_grid_ids) AND year >= 2009 AND year <= 2020 AND ARRAY_LENGTH(category_sdg) > 0
  ) as total_sdg_pubs,

  -- Citations between 2009 and 2020
  (SELECT SUM(citations_count) FROM all_pubs) as total_pub_citations,
  
  (SELECT AVG(citations_count) FROM all_pubs) as avg_pub_citations,
  
  (SELECT SUM(patent_citations) FROM all_pubs) as total_pub_patent_citations,
  
  -- Grants between 2009 and 2020
  (SELECT COUNT(*) FROM grants) as total_grants,
  
  (SELECT SUM(funding_amount_in_usd) FROM grants) as total_grants_amount_usd,
  
  -- Patents
  (SELECT COUNT(*) FROM patents) as total_patents,

  (SELECT IF(SUM(pub_citations) > 0, SUM(pub_citations), 0) FROM patents) as total_patent_citations,


  -- Clinical Trials
  (SELECT * FROM clinical_trails_count) as total_clinical_trails,

  -- Data Sets
  (SELECT * FROM data_sets_count) as total_data_sets,


  -- Total (2009)
  (SELECT * FROM pubs_count_2009) as total_2009,
  
  -- OA (2009)
  (SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2009 AND pubs.is_oa) as total_oa_2009,
  
  -- OA percentage (2009)
  (SELECT IF(
    (SELECT * FROM pubs_count_2009) > 0,
    COUNT(DISTINCT pubs.id) / (SELECT * FROM pubs_count_2009),
    0)
  FROM pubs WHERE pubs.year = 2009 AND pubs.is_oa) as percent_oa_2009,

  -- Total (2019)
  (SELECT * FROM pubs_count_2019) as total_2019,
  
  -- OA (2019)
  (SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2019 AND pubs.is_oa) as total_oa_2019,
  
  -- OA percentage (2019)
  (SELECT IF(
    (SELECT * FROM pubs_count_2019) > 0,
    COUNT(DISTINCT pubs.id) / (SELECT * FROM pubs_count_2019),
    0)
  FROM pubs WHERE pubs.year = 2019 AND pubs.is_oa) as percent_oa_2019,


  -- Pre-prints (2009)
  (SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2009 AND pubs.is_preprint) as total_preprint_2009,

  -- Pre-prints (2019)
  (SELECT COUNT(DISTINCT pubs.id) FROM pubs WHERE pubs.year = 2019 AND pubs.is_preprint) as total_preprint_2019,


  -- International Collab (2009)
  (IF(
    (SELECT * FROM pubs_count_2009) > 0, 
    ((
      SELECT 
        COUNT(DISTINCT pubs.id)
      FROM 
        pubs
      INNER JOIN
        (SELECT DISTINCT id FROM `dimensions-ai.data_analytics.grid`, UNNEST(addresses) a WHERE a.country_code not in UNNEST(
          (SELECT ARRAY(SELECT a.country_code FROM `dimensions-ai.data_analytics.grid`, UNNEST(addresses) a WHERE id = @gridid))
        )) grid
        ON pubs.author_grid = grid.id
      WHERE 
        pubs.author_grid != @gridid AND pubs.year = 2009
    ) / (SELECT * FROM pubs_count_2009))
    , 0
  )) as int_collab_2009,
  
  -- International Collab (2019)
  (IF(
    (SELECT * FROM pubs_count_2019) > 0, 
    ((
      SELECT 
        COUNT(DISTINCT pubs.id)
      FROM 
        pubs
      INNER JOIN
        (SELECT DISTINCT id FROM `dimensions-ai.data_analytics.grid`, UNNEST(addresses) a WHERE a.country_code not in UNNEST(
          (SELECT ARRAY(SELECT a.country_code FROM `dimensions-ai.data_analytics.grid`, UNNEST(addresses) a WHERE id = @gridid))
        )) grid
        ON pubs.author_grid = grid.id
      WHERE 
        pubs.author_grid != @gridid AND pubs.year = 2019
    ) / (SELECT * FROM pubs_count_2019))
    , 0
  )) as int_collab_2019


total_pubs  total_sdg_pubs  ...  int_collab_2009  int_collab_2019
0      130284           15063  ...         0.417995         0.576052

[1 rows x 21 columns]

## (B) Org SDG Report - publications per SDG per year

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH pubs AS (
    SELECT 
      id, 
      year, 
      sus_dev_goals.name as sdg_name, 
      author_grid_ids
    FROM 
      `dimensions-ai.data_analytics.publications`, UNNEST(category_sdg) sus_dev_goals
    WHERE
      ARRAY_LENGTH(category_sdg) > 0
      AND
      year > (EXTRACT(YEAR FROM CURRENT_DATE()) - 10)
)

SELECT 
  COUNT(*) as total, year, sdg_name, "Global" as type
FROM pubs 
GROUP BY year, sdg_name

UNION ALL

SELECT 
  COUNT(*) as count, year, sdg_name, "Inst" as type
FROM pubs 
WHERE @gridid IN UNNEST(author_grid_ids)
GROUP BY year, sdg_name

total  year                                   sdg_name    type
0    24791  2013                        4 Quality Education  Global
1     2568  2015                               1 No Poverty  Global
2    11338  2020          8 Decent Work and Economic Growth  Global
3     8093  2011      11 Sustainable Cities and Communities  Global
4       67  2016          8 Decent Work and Economic Growth    Inst
..     ...   ...                                        ...     ...
331   1023  2018              17 Partnerships for the Goals  Global
332   3444  2018                               1 No Poverty  Global
333   2956  2012               6 Clean Water and Sanitation  Global
334   5302  2020  12 Responsible Consumption and Production  Global
335   1061  2020  9 Industry, Innovation and Infrastructure  Global

[336 rows x 4 columns]

## (C) Org SDG Report - global average deviations

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH pubs AS (
  SELECT 
    id, 
    year, 
    sus_dev_goals.name as sdg_name, 
    author_grid_ids
  FROM 
    `dimensions-ai.data_analytics.publications`, UNNEST(category_sdg) sus_dev_goals
  WHERE 
    year >= 2009 AND year <= 2020 AND ARRAY_LENGTH(category_sdg) > 0
)

SELECT 
	global.sdg_name, 
	sdg_index, 
	percentage, 
	int_count, 
	int_total, 
	ROUND(int_count/int_total, 4) as int_percentage 
FROM (
  SELECT sdg_name, ROUND(AVG(count/total), 4) as percentage
  FROM
    (
      SELECT
        grid,
        sdg_name,
        COUNT(DISTINCT p.id) as count
      FROM
        pubs p, UNNEST(p.author_grid_ids) grid
      GROUP BY
        grid, sdg_name
    ) per_grid_count
  LEFT JOIN
    (
      SELECT
        grid,
        COUNT(DISTINCT pp.id) as total
      FROM 
        pubs pp, UNNEST(pp.author_grid_ids) grid
      GROUP BY
        grid
    ) per_grid_total
  ON per_grid_count.grid = per_grid_total.grid
  WHERE
    per_grid_total.total > 200
  GROUP BY sdg_name
) global

LEFT JOIN 
  (
    SELECT 
      sdg_name,
      CAST(SPLIT(sdg_name, " ")[OFFSET(0)] as INT64) as sdg_index,
      (SELECT COUNT(DISTINCT id) FROM pubs WHERE sdg_name = p.sdg_name AND @gridid IN UNNEST(author_grid_ids)) as int_count,
      (SELECT COUNT(DISTINCT id) FROM pubs WHERE @gridid IN UNNEST(author_grid_ids)) as int_total,
    FROM pubs p
    GROUP BY sdg_name 
  ) int_grps

ON global.sdg_name = int_grps.sdg_name

sdg_name  ...  int_percentage
0                         4 Quality Education  ...          0.0439
1               7 Affordable and Clean Energy  ...          0.1251
2                           13 Climate Action  ...          0.1277
3                3 Good Health and Well Being  ...          0.3944
4                         14 Life Below Water  ...          0.0094
5   12 Responsible Consumption and Production  ...          0.0060
6               17 Partnerships for the Goals  ...          0.0017
7       11 Sustainable Cities and Communities  ...          0.0192
8   16 Peace, Justice and Strong Institutions  ...          0.1482
9                           5 Gender Equality  ...          0.0053
10               6 Clean Water and Sanitation  ...          0.0107
11                              2 Zero Hunger  ...          0.0270
12                            15 Life on Land  ...          0.0297
13                    10 Reduced Inequalities  ...          0.0524
14          8 Decent Work and Economic Growth  ...          0.0402
15                               1 No Poverty  ...          0.0147
16  9 Industry, Innovation and Infrastructure  ...          0.0029

[17 rows x 6 columns]

## (D) Org SDG Report - top authors

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH pubs AS (
  SELECT 
    author as researcher_id,
    s.name as sdg,
    COUNT(*) as pub_count,
    ROW_NUMBER() OVER ( PARTITION BY s.name ORDER BY COUNT(*) desc ) as rank
  FROM `dimensions-ai.data_analytics.publications` p,
    UNNEST(author_researcher_ids) author,
    UNNEST(category_sdg) s
  WHERE 
    @gridid in UNNEST(author_grid_ids) 
      AND year >= 2009 
      AND ARRAY_LENGTH(p.category_sdg) > 0 
  GROUP BY
    author, s.name
  ORDER BY pub_count desc
)

SELECT 
  researcher_id, 
  p.pub_count as sdg_pub_count,
  counts.total_pub_count as total_pub_count,
  sdg,
  CONCAT(IF(rsch.first_name is not null, rsch.first_name, ""), IF(rsch.first_name is not null, " ", ""), IF(rsch.last_name is not null, rsch.last_name, "")) as author_name,
  rsch.current_org as current_org_id,
  grid.name as current_org_name,
  IF(rsch.current_org = @gridid, true, false) as currently_affiliated
FROM pubs p
LEFT JOIN `dimensions-ai.data_analytics.researchers` rsch
  ON rsch.id = researcher_id
LEFT JOIN 
  (
    SELECT rsch_id, COUNT(*) as total_pub_count 
    FROM `dimensions-ai.data_analytics.publications`, UNNEST(author_researcher_ids) rsch_id
    WHERE year >= 2009
    GROUP BY rsch_id
  ) counts
  ON counts.rsch_id = researcher_id
LEFT JOIN
  `dimensions-ai.data_analytics.grid` grid
  ON rsch.current_org = grid.id
WHERE rank < 6
ORDER BY sdg, sdg_pub_count desc


researcher_id  ...  currently_affiliated
0    ur.01204305732.85  ...                  True
1   ur.012657145676.08  ...                  True
2     ur.0743253460.13  ...                 False
3    ur.01321440243.14  ...                 False
4     ur.0732052166.01  ...                 False
..                 ...  ...                   ...
80    ur.0653450630.42  ...                  True
81  ur.013270326357.77  ...                  True
82  ur.011675365357.28  ...                  True
83  ur.012265056757.11  ...                  True
84  ur.010543463515.86  ...                 False

[85 rows x 8 columns]

## (E) Org SDG Report - recent publications

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH pubs AS (
  SELECT DISTINCT
    id, 
    year,
    publication_date,
    citations_count, 
    title.preferred as title, 
    publication_type, 
    sus_dev_goals.name as sdg_name, 
    sus_dev_goals.code as sdg_code
  FROM `dimensions-ai.data_analytics.publications`, UNNEST(category_sdg) sus_dev_goals
  WHERE year > (EXTRACT(YEAR FROM CURRENT_DATE()) - 10) AND @gridid IN UNNEST(author_grid_ids)
)

SELECT * FROM pubs 
ORDER BY year desc, publication_date desc 
LIMIT 100

id  year  ...                                   sdg_name  sdg_code
0   pub.1125085467  2020  ...               3 Good Health and Well Being     40003
1   pub.1127156575  2020  ...               3 Good Health and Well Being     40003
2   pub.1124456050  2020  ...               3 Good Health and Well Being     40003
3   pub.1124952136  2020  ...               3 Good Health and Well Being     40003
4   pub.1127864300  2020  ...               3 Good Health and Well Being     40003
..             ...   ...  ...                                        ...       ...
95  pub.1125025168  2020  ...               3 Good Health and Well Being     40003
96  pub.1125132736  2020  ...               3 Good Health and Well Being     40003
97  pub.1128301084  2020  ...               3 Good Health and Well Being     40003
98  pub.1126056546  2020  ...  12 Responsible Consumption and Production     40012
99  pub.1124228441  2020  ...               3 Good Health and Well Being     40003

[100 rows x 8 columns]

## (F) Org SDG Report - funding overview

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH grants AS (
  SELECT id, CAST(start_date as DATE) as start_date, CAST(end_date as DATE) as end_date, funding_amount_in_usd
  FROM `dimensions-ai.data_analytics.grants` 
  WHERE @gridid in UNNEST(research_organisations)
)

SELECT 
  (SELECT COUNT(*) FROM grants WHERE EXTRACT(YEAR from start_date) <= 2020 AND EXTRACT(YEAR from end_date) >= 2020) as active_count,
  (SELECT COUNT(*) FROM grants WHERE EXTRACT(YEAR from start_date) >= 2009 AND EXTRACT(YEAR from start_date) <= 2020) as total_count,

  (SELECT SUM(funding_amount_in_usd) FROM grants WHERE EXTRACT(YEAR from start_date) <= 2020 AND EXTRACT(YEAR from end_date) >= 2020) as active_funding,
  (SELECT SUM(funding_amount_in_usd) FROM grants WHERE EXTRACT(YEAR from start_date) >= 2009 AND EXTRACT(YEAR from start_date) <= 2020) as total_funding


active_count  total_count  active_funding  total_funding
0          3249         9920    5.759956e+09   1.332133e+10

## (G) Org SDG Report - yearly grants summary

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH grants AS (
  SELECT 
  *,
  GENERATE_ARRAY(start_year, end_year, 1) as active_years
  FROM (
    SELECT 
      id, 
      CAST(start_date as DATE) as start_date, 
      CAST(end_date as DATE) as end_date, 
      funding_amount_in_usd, 
      start_year,
      EXTRACT(YEAR FROM CAST(end_date as DATE)) as end_year
    FROM `dimensions-ai.data_analytics.grants` 
    WHERE @gridid in UNNEST(research_organisations)
  )
)

SELECT 
  active_year as year,
  COUNT(*) as active_grants,
  SUM(IF(g.start_year=active_year, 1, 0)) as starting_grants,
  SUM(g.funding_amount_in_usd) as active_funding_amount_in_usd,
  SUM(IF(g.start_year=active_year, g.funding_amount_in_usd, 0)) as starting_funding_amount_in_usd
FROM 
  grants g, 
  UNNEST(active_years) active_year
WHERE
  active_year >= 2009 AND active_year <= 2020
  
GROUP BY active_year
ORDER BY active_year

year  ...  starting_funding_amount_in_usd
0   2009  ...                    1.039871e+09
1   2010  ...                    1.024812e+09
2   2011  ...                    1.156044e+09
3   2012  ...                    1.151052e+09
4   2013  ...                    1.476341e+09
5   2014  ...                    1.220906e+09
6   2015  ...                    1.467130e+09
7   2016  ...                    1.298187e+09
8   2017  ...                    1.101020e+09
9   2018  ...                    1.055138e+09
10  2019  ...                    8.898644e+08
11  2020  ...                    3.660451e+08

[12 rows x 5 columns]

## (H) Org SDG Report - recent top funders

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH grants AS (
  SELECT 
  *,
  GENERATE_ARRAY(start_year, end_year, 1) as active_years,
  IF(start_year<=2020 AND end_year>=2020, 1, 0) as is_active,
  FROM (
    SELECT 
      id,
      funder,
      CAST(start_date as DATE) as start_date, 
      CAST(end_date as DATE) as end_date, 
      funding_amount_in_usd, 
      start_year,
      EXTRACT(YEAR FROM CAST(end_date as DATE)) as end_year
    FROM 
    	`dimensions-ai.data_analytics.grants`
    WHERE
    	@gridid in UNNEST(research_organisations) 
  )
)

SELECT 
  grid.name as funder_name, 
  t.*,
  (SELECT SUM(is_active) FROM grants  WHERE funder = t.funder) as active_grants
FROM (
  SELECT 
    g.funder as funder,
    COUNT(*) as total_grants
  FROM 
    grants g
  WHERE
    g.start_year >= 2009 AND g.start_year <= 2020
  GROUP BY g.funder
  
) t
LEFT JOIN
  `dimensions-ai.data_analytics.grid` grid
ON t.funder = grid.id

ORDER BY total_grants desc

funder_name  ... active_grants
0    Engineering and Physical Sciences Research Cou...  ...           785
1                                       Wellcome Trust  ...           348
2                             Medical Research Council  ...           388
3                                  European Commission  ...           256
4    Biotechnology and Biological Sciences Research...  ...           312
..                                                 ...  ...           ...
124                NIHR Central Commissioning Facility  ...             0
125  National Institute on Deafness and Other Commu...  ...             1
126                                      Autism Speaks  ...             0
127                                     Meningitis Now  ...             0
128                            British Lung Foundation  ...             0

[129 rows x 4 columns]

## (I) Org SDG Report - recent grants

In [ ]:
%%bigquery --params $bq_params --project $project_id 
WITH grants AS (
  SELECT 
    id,
    title,
    CAST(start_date as DATE) as start_date, 
    CAST(end_date as DATE) as end_date,
    funder,
    funding_amount_in_usd
  FROM `dimensions-ai.data_analytics.grants` 
  WHERE 
    @gridid in UNNEST(research_organisations)
    AND EXTRACT(YEAR FROM CAST(start_date as DATE)) <= EXTRACT(YEAR FROM CURRENT_DATE())
    AND 
      (EXTRACT(YEAR FROM CAST(end_date as DATE)) >= EXTRACT(YEAR FROM CURRENT_DATE()) OR end_date IS NULL)
  ORDER BY
    start_date desc, end_date desc
  LIMIT 50
)

SELECT 
  g.id,
  g.title,
  g.start_date,
  g.end_date,
  grid.name,
  g.funder,
  g.funding_amount_in_usd,
FROM grants g
LEFT JOIN `dimensions-ai.data_general.grid` grid
  ON grid.id = g.funder

id  ... funding_amount_in_usd
0   grant.9054862  ...             7762353.0
1   grant.9064801  ...              237710.0
2   grant.9064643  ...              237710.0
3   grant.8963591  ...             2198288.0
4   grant.8964408  ...             2516349.0
5   grant.8558602  ...              680259.0
6   grant.8964487  ...              251104.0
7   grant.9064893  ...              251104.0
8   grant.9064506  ...              237710.0
9   grant.9054541  ...             1611961.0
10  grant.9054693  ...              410300.0
11  grant.9054513  ...              572087.0
12  grant.9054687  ...              548219.0
13  grant.9054609  ...              794019.0
14  grant.8673697  ...             1498356.0
15  grant.8964067  ...             2192494.0
16  grant.8709335  ...             1674567.0
17  grant.8963710  ...             2212016.0
18  grant.9065861  ...             2326372.0
19  grant.8709667  ...             1673951.0
20  grant.8857844  ...             2510435.0
21  grant.9064583  ...              251102.0
22  grant.9064435  ...              237709.0
23  grant.9064591  ...              237709.0
24  grant.9064626  ...              237709.0
25  grant.8584950  ...              237709.0
26  grant.9065117  ...              237709.0
27  grant.9064826  ...              251102.0
28  grant.8389152  ...              237709.0
29  grant.8709787  ...              167452.0
30  grant.9054515  ...             1365204.0
31  grant.9065734  ...             6586674.0
32  grant.9064456  ...              237710.0
33  grant.8709665  ...             1673090.0
34  grant.8964099  ...             2232598.0
35  grant.9065407  ...             8030136.0
36  grant.9064909  ...              237710.0
37  grant.8963659  ...              237710.0
38  grant.8964442  ...              251104.0
39  grant.8857215  ...              400287.0
40  grant.9054876  ...              386713.0
41  grant.8671583  ...              686072.0
42  grant.8964019  ...             2155950.0
43  grant.9065401  ...             6495020.0
44  grant.8964425  ...              251102.0
45  grant.9064440  ...              237710.0
46  grant.9065711  ...              237710.0
47  grant.9064513  ...              251102.0
48  grant.9064873  ...              237710.0
49  grant.8709952  ...             1669033.0

[50 rows x 7 columns]

## (J) Org SDG Report - top cited publications

In [ ]:
%%bigquery --params $bq_params --project $project_id 
SELECT id, title, journal.title as jtitle, citations_count 
FROM `dimensions-ai.data_analytics.publications` 
WHERE @gridid in UNNEST(author_grid_ids) and year >= 2011 and year <= 2020
ORDER BY citations_count desc
LIMIT 3


id  ... citations_count
0  pub.1018963471  ...           15886
1  pub.1041893471  ...           10032
2  pub.1013705877  ...            7990

[3 rows x 4 columns]

## (K) Org SDG Report - top altmetric score publications

In [ ]:
%%bigquery --params $bq_params --project $project_id 
SELECT id, title, journal.title as jtitle, altmetrics.score 
FROM `dimensions-ai.data_analytics.publications` 
WHERE @gridid in UNNEST(author_grid_ids) and year = 2020
ORDER BY altmetrics.score desc
LIMIT 3

id  ...  score
0  pub.1127239818  ...  12426
1  pub.1126016857  ...   7919
2  pub.1125492157  ...   5314

[3 rows x 4 columns]